<a href="https://colab.research.google.com/github/HemanthNarayanadas/Performance-comparison-of-CNN-optimizer-Sgd-vs-Adam/blob/main/CNN_OPTIMIZER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import files
files.upload()  # ⬅️ Upload your kaggle.json here


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"parepallysaivarshith","key":"27fd11b12e7583945257a34540d33230"}'}

In [6]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [10]:

!kaggle datasets download -d gpiosenka/sports-classification


Dataset URL: https://www.kaggle.com/datasets/gpiosenka/sports-classification
License(s): CC0-1.0
 94% 397M/424M [00:00<00:00, 364MB/s]
100% 424M/424M [00:01<00:00, 403MB/s]


In [11]:
# Step 4 — Unzip it
!unzip sports-classification.zip -d dataset


Streaming output truncated to the last 5000 lines.
  inflating: dataset/train/rings/022.jpg  
  inflating: dataset/train/rings/023.jpg  
  inflating: dataset/train/rings/024.jpg  
  inflating: dataset/train/rings/025.jpg  
  inflating: dataset/train/rings/026.jpg  
  inflating: dataset/train/rings/027.jpg  
  inflating: dataset/train/rings/028.jpg  
  inflating: dataset/train/rings/029.jpg  
  inflating: dataset/train/rings/030.jpg  
  inflating: dataset/train/rings/031.jpg  
  inflating: dataset/train/rings/032.jpg  
  inflating: dataset/train/rings/033.jpg  
  inflating: dataset/train/rings/034.jpg  
  inflating: dataset/train/rings/035.jpg  
  inflating: dataset/train/rings/036.jpg  
  inflating: dataset/train/rings/037.jpg  
  inflating: dataset/train/rings/038.jpg  
  inflating: dataset/train/rings/039.jpg  
  inflating: dataset/train/rings/040.jpg  
  inflating: dataset/train/rings/041.jpg  
  inflating: dataset/train/rings/042.jpg  
  inflating: dataset/train/rings/043.jpg  
  i

In [ ]:
# ============================================================
# CNN OPTIMIZER COMPARISON (SGD vs ADAM)
# Works with any folder-structured dataset
# ============================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time, warnings
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

warnings.filterwarnings("ignore")

# ============================================================
# STEP 1: DATASET LOADING & PREPROCESSING
# ============================================================
DATA_DIR = "/content/dataset"  # 🔹 Change path if your dataset is elsewhere
IMG_SIZE = (64, 64)
BATCH_SIZE = 32
EPOCHS = 5
SEED = 42

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_gen = datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    seed=SEED
)

val_gen = datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    seed=SEED
)

# 🔹 Print class info
class_labels = list(train_gen.class_indices.keys())
num_classes = len(class_labels)
print("\n✅ Dataset Loaded Successfully!")
print(f"📦 Number of Classes: {num_classes}")
print(f"🏷️ Class Names: {class_labels}\n")

# ============================================================
# STEP 2: MODEL DEFINITION
# ============================================================
def build_cnn(input_shape=(64, 64, 3), num_classes=10, wd=1e-4):
    model = models.Sequential([
        layers.Conv2D(32, (3,3), padding='same', activation='relu',
                      kernel_regularizer=regularizers.l2(wd), input_shape=input_shape),
        layers.MaxPooling2D(2,2),
        layers.Conv2D(64, (3,3), activation='relu', padding='same'),
        layers.MaxPooling2D(2,2),
        layers.Conv2D(128, (3,3), activation='relu', padding='same'),
        layers.MaxPooling2D(2,2),
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

# ============================================================
# STEP 3: TRAINING WITH SGD & ADAM
# ============================================================
optimizers = {
    "SGD": tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9),
    "Adam": tf.keras.optimizers.Adam(learning_rate=0.001)
}

results = {}
histories = {}

for opt_name, optimizer in optimizers.items():
    print(f"\n🚀 Training with {opt_name} optimizer...\n")

    model = build_cnn(input_shape=(64,64,3), num_classes=num_classes)
    model.compile(optimizer=optimizer,
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])

    start = time.time()
    history = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=EPOCHS,
        verbose=2
    )
    end = time.time()

    val_loss, val_acc = model.evaluate(val_gen, verbose=0)
    results[opt_name] = {"Validation Accuracy": val_acc, "Training Time (s)": end - start}
    histories[opt_name] = history.history

# ============================================================
# STEP 4: PLOT ACCURACY & LOSS
# ============================================================
plt.figure(figsize=(10,5))
for opt in histories:
    plt.plot(histories[opt]['val_accuracy'], label=f"{opt} Validation Accuracy")
plt.title("Validation Accuracy: SGD vs Adam")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.grid(True)
plt.show()

plt.figure(figsize=(10,5))
for opt in histories:
    plt.plot(histories[opt]['val_loss'], label=f"{opt} Validation Loss")
plt.title("Validation Loss: SGD vs Adam")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)
plt.show()

# ============================================================
# STEP 5: SUMMARY
# ============================================================
summary_df = pd.DataFrame(results).T
print("\n📊 Final Results Summary:")
print(summary_df)

summary_df.to_csv("optimizer_results.csv", index=True)
print("\n✅ Training complete — results saved as optimizer_results.csv")


Found 11594 images belonging to 3 classes.
Found 2898 images belonging to 3 classes.

✅ Dataset Loaded Successfully!
📦 Number of Classes: 3
🏷️ Class Names: ['test', 'train', 'valid']


🚀 Training with SGD optimizer...

Epoch 1/5
363/363 - 136s - 375ms/step - accuracy: 0.9301 - loss: 0.3211 - val_accuracy: 0.9310 - val_loss: 0.3023
Epoch 2/5
363/363 - 140s - 384ms/step - accuracy: 0.9310 - loss: 0.3072 - val_accuracy: 0.9310 - val_loss: 0.2997
Epoch 3/5
363/363 - 133s - 367ms/step - accuracy: 0.9310 - loss: 0.3059 - val_accuracy: 0.9310 - val_loss: 0.3028
Epoch 4/5
